<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Natural Language Processing (NLP)
## *Data Science Unit 4 Sprint 2 Assignment 1*

Analyze a corpus of text using text visualization of token frequency. Try cleaning the data as much as possible. Try the following techniques: 
- Stemming
- Lemmatization
- Custom stopword removal
- Using frequency based stopwrod removal

You are free to use any dataset you are interested in. Kaggle is a great place to start. Feel free to sample the data if the dataset is too large to hanlde in memory. 

In [2]:
# get enron dataset
# !wget https://www.kaggle.com/wcukierski/enron-email-dataset/downloads/enron-email-dataset.zip/2

In [1]:
"""
Import Statements
"""

# Base
from collections import Counter
import re
import pandas as pd

# Plotting
import squarify
import matplotlib.pyplot as plt
import seaborn as sns

# NLP Libraries
import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer

nlp = spacy.load("en_core_web_sm")

In [2]:
emails = pd.read_csv('./data/emails.csv')

In [3]:
emails.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [18]:
emails.message.loc[9].split('\n')

['Message-ID: <30795301.1075855687494.JavaMail.evans@thyme>',
 'Date: Mon, 16 Oct 2000 06:44:00 -0700 (PDT)',
 'From: phillip.allen@enron.com',
 'To: zimam@enron.com',
 'Subject: FW: fixed forward or other Collar floor gas price terms',
 'Mime-Version: 1.0',
 'Content-Type: text/plain; charset=us-ascii',
 'Content-Transfer-Encoding: 7bit',
 'X-From: Phillip K Allen',
 'X-To: zimam@enron.com',
 'X-cc: ',
 'X-bcc: ',
 "X-Folder: \\Phillip_Allen_Dec2000\\Notes Folders\\'sent mail",
 'X-Origin: Allen-P',
 'X-FileName: pallen.nsf',
 '',
 '---------------------- Forwarded by Phillip K Allen/HOU/ECT on 10/16/2000 ',
 '01:42 PM ---------------------------',
 '',
 '',
 '"Buckner, Buck" <buck.buckner@honeywell.com> on 10/12/2000 01:12:21 PM',
 'To: "\'Pallen@Enron.com\'" <Pallen@Enron.com>',
 'cc:  ',
 'Subject: FW: fixed forward or other Collar floor gas price terms',
 '',
 '',
 'Phillip,',
 '',
 '> As discussed  during our phone conversation, In a Parallon 75 microturbine',
 '> power generatio

In [5]:
# cleaning data courtesty of https://towardsdatascience.com/how-i-used-machine-learning-to-classify-emails-and-turn-them-into-insights-efed37c1e66
# no thanks to  https://towardsdatascience.com/@anthonydm as this is an absolutely terrible approach

def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email

def parse_into_emails(messages):
    emails = [parse_raw_message(message) for message in messages]
    
    return {
        'body': [email['body'] for email in emails], 
        'to': [email['to'] for email in emails], 
        'from': [email['from'] for email in emails]
    }

# email_df = pd.DataFrame(parse_into_emails(emails.message))


In [7]:
parse_into_emails(emails.message)

KeyError: 'to'

In [58]:
sample = emails.sample(5)

In [96]:
sample[-1:]['message'].tolist()

['Message-ID: <9702587.1075841867339.JavaMail.evans@thyme>\nDate: Thu, 14 Dec 2000 18:57:00 -0800 (PST)\nFrom: pmadpr@worldnet.att.net\nSubject: PowerMarketers.com Daily Power Report for 15 December 2000\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: "The Power Marketing Association" <pmadpr@worldnet.att.net>\nX-To: \nX-cc: \nX-bcc: \nX-Folder: \\kate symes 6-27-02\\Notes Folders\\Power marketer\nX-Origin: SYMES-K\nX-FileName: kate symes 6-27-02.nsf\n\nAttention POWER REPORT Readers: Go to \nhttp://www.powermarketers.com/prdaily.htm to view today\'s edition with direct \nheadline links to each story.]\n\n======================================================================\nThe Industry\'s Preferred Source For Breaking Energy News And Information\nhttp://www.powermarketers.com\n----------------------------------------------------------------------\nT H E P O W E R M A R K E T E R S .C O M P O W E R R E P O R T\n\nToday\'s Headl

In [79]:
test = parse_into_emails(sample.message)

In [81]:
test[4]

{'from': 'pmadpr@worldnet.att.net',
 'body': 'headline links to each story.]======================================================================The Industry\'s Preferred Source For Breaking Energy News And Information----------------------------------------------------------------------T H E P O W E R M A R K E T E R S .C O M P O W E R R E P O R TToday\'s Headlines From POWERMARKETERS.COMSubscriptions to The PowerMarketers.com Power Report are free - you willnever be charged.You may redistribute this e-mail without change. You may not redistributestories.************************************************************************------Advertisement------THE ENERGY INFO SOURCE TRANSMISSION REPORT (December 2000 Edition)* Detailed Analysis of October 15th RTO and ISO Filings* Transcos and Gridcos* Maps of Transco and RTO areas* Much, much more.FINANCIAL TIMES NORTH AMERICAN ELECTRIC POWER SYSTEM WALL MAPSpecial Holiday Offer-- $100 Off if Ordered by New Years!******************************

In [76]:
[email['to'] for email in test]

KeyError: 'to'

In [72]:
{'body': [email['body'] for email in test], 
        'to': [email['to'] for email in test], 
        'from': [email['from'] for email in test]}

KeyError: 'to'

In [ ]:
# first I will tokenize
import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_sm")

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

## Stretch Goals

* Write a web scraper that can scrape "Data Scientist" job listings from indeed.com.
* Look ahead to some of the topics from later this week:
 - Part of Speech Tagging
 - Named Entity Recognition
 - Document Classification
* Try a different visualization techniques
* Automate the process of retriving job listings. ;)